# Make TBL dark, open beam and sample images from Ymir data

This notebook creates files for TBL that contain images in the Orca detector file entry.
We used a TBL file written by ECDC as a template (`857127_00000212.hdf`) and replaced the orca detector data with images recorded at Ymir.

The Ymir data was a single file containing dark frame, open beam, and sample data.
We split this into 3 separate datasets and files, as this is the way TBL would like to operate (at least initially).

We then add an arbitrary exposure time entry in the file for the images.
Finally, we generate a fake proton-charge log and place it inside the `neutron_prod_info` group.
The proton charge is a time series from the first frame time to the last frame time + exposure time, where the value is randomly oscillating around 1.

In [ ]:
import h5py as h5
import numpy as np
import scipp as sc

In [ ]:
from ess.ymir.data import ymir_lego_images_path

## Load Ymir data

In [ ]:
path = 'entry/instrument/histogram_mode_detectors/orca/data/'

with h5.File(ymir_lego_images_path()) as f:
    lego_data = f[path+'value'][()]
    lego_time = f[path+'time'][()]

## Split into dark, open beam, and sample

In [ ]:
dark_data = lego_data[:5, ...]
dark_time = lego_time[:5, ...]

ob_data = lego_data[5:10, ...]
ob_time = lego_time[5:10, ...]

sample_data = lego_data[10:, ...]
sample_time = lego_time[10:, ...]

## Create fake proton charge

In [ ]:
exposure_time = int(5e8) # 0.5s (made up)
period = 1e9/14

dark_charge_time = np.arange(float(dark_time[0]), float(dark_time[-1] + exposure_time), period).astype('uint64')
dark_charge_value = np.random.uniform(0.99, 1.01, size=len(dark_charge_time)).astype('float64')

ob_charge_time = np.arange(float(ob_time[0]), float(ob_time[-1] + exposure_time), period).astype('uint64')
ob_charge_value = np.random.uniform(0.99, 1.01, size=len(ob_charge_time)).astype('float64')

sample_charge_time = np.arange(float(sample_time[0]), float(sample_time[-1] + exposure_time), period).astype('uint64')
sample_charge_value = np.random.uniform(0.99, 1.01, size=len(sample_charge_time)).astype('float64')

## Make pixel offsets

In [ ]:
npix = dark_data.shape[-1]

x_pixel_offset = sc.linspace('dim_1', -0.1, 0.1, npix, unit='m')
y_pixel_offset = sc.linspace('dim_2', -0.1, 0.1, npix, unit='m')
x_pixel_offset

## Create new files from template

In [ ]:
def replace_dataset(entry, name, values):
    attrs = dict(entry[name].attrs)
    del entry[name]
    dset = entry.create_dataset(name, data=values)
    dset.attrs.update(attrs)

def make_new_file(template_nexus_file, outfile, data, proton_charge):
    import shutil
    shutil.copyfile(template_nexus_file, outfile)

    # detector_path = 'entry/instrument/histogram_mode_detectors/orca'  # ODIN
    detector_path = 'entry/instrument/orca_detector/'  # TBL
    proton_charge_path = 'entry/neutron_prod_info/pulse_charge'
    
    with h5.File(outfile, "r+") as f:

        detector_data = f[detector_path+"data"]
        replace_dataset(detector_data, name="value", values=data["value"])
        replace_dataset(detector_data, name="time", values=data["time"])

        detector = f[detector_path]
        detector.copy('intensifier', detector, 'camera_exposure')
        exp = f[detector_path+"camera_exposure"]
        replace_dataset(exp, name="value", values=np.array([exposure_time]))
        replace_dataset(exp, name="average_value", values=exposure_time)
        replace_dataset(exp, name="minimum_value", values=exposure_time)
        replace_dataset(exp, name="maximum_value", values=exposure_time)
        f[detector_path+"camera_exposure/value"].attrs['units'] = 'ns'
        f[detector_path+"camera_exposure/average_value"].attrs['units'] = 'ns'
        f[detector_path+"camera_exposure/minimum_value"].attrs['units'] = 'ns'
        f[detector_path+"camera_exposure/maximum_value"].attrs['units'] = 'ns'

        xoff = detector.create_dataset('x_pixel_offset', data=x_pixel_offset.values)
        xoff.attrs['units'] = str(x_pixel_offset.unit)
        xoff.attrs['axis'] = 1
        yoff = detector.create_dataset('y_pixel_offset', data=y_pixel_offset.values)
        yoff.attrs['units'] = str(y_pixel_offset.unit)
        yoff.attrs['axis'] = 2
        detector.attrs['axes'] = ['dim_2', 'dim_1']

        pcharge = f[proton_charge_path]
        replace_dataset(pcharge, name="value", values=proton_charge["value"])
        replace_dataset(pcharge, name="time", values=proton_charge["time"])
        replace_dataset(pcharge, name="average_value", values=proton_charge["value"].mean())
        replace_dataset(pcharge, name="minimum_value", values=proton_charge["value"].min())
        replace_dataset(pcharge, name="maximum_value", values=proton_charge["value"].max())

In [ ]:
template_nexus_file = "857127_00000212.hdf"

# Dark run
make_new_file(
    template_nexus_file=template_nexus_file,
    outfile="ymir_lego_dark_run.hdf",
    data={"value": dark_data, "time": dark_time},
    proton_charge={"value": dark_charge_value, "time": dark_charge_time}
)

In [ ]:
# Openbeam run
make_new_file(
    template_nexus_file=template_nexus_file,
    outfile="ymir_lego_openbeam_run.hdf",
    data={"value": ob_data, "time": ob_time},
    proton_charge={"value": ob_charge_value, "time": ob_charge_time}
)

In [ ]:
# Sample run
make_new_file(
    template_nexus_file=template_nexus_file,
    outfile="ymir_lego_sample_run.hdf",
    data={"value": sample_data, "time": sample_time},
    proton_charge={"value": sample_charge_value, "time": sample_charge_time}
)